# 6th attempt - RCNN

In [1]:
import numpy as np
import pandas as pd
from functions import *
from read_from_file_df import *
import pickle
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [ ]:
SIZE = 10
AMOUNT_BOARDS = 10000

In [3]:
gen = 2
name_df = f'{PATH_DF}\\{SIZE}-{AMOUNT_BOARDS}\\{SIZE}size_{AMOUNT_BOARDS}boards_{gen}gen_reverse'
reverse_df = pd.read_pickle(f'{name_df}.pkl')

In [4]:
new_columns = [f'Col_{i}' for i in range(gen*SIZE*SIZE)]
reverse_df_sort = reverse_df.sort_values(by = new_columns).reset_index(drop=True)
for i in reverse_df_sort.columns:
    reverse_df_sort[i] = reverse_df_sort[i].astype(int)

In [5]:
print("reverse df:", len(reverse_df))
print("reverse df sort:",len(reverse_df_sort))

reverse df: 41366
reverse df sort: 41366


In [6]:
# Step 1: Prepare Data
amount_features = len(reverse_df_sort.columns) - SIZE*SIZE #the previous boards
features = reverse_df_sort.iloc[:, :amount_features]
name_col = 'Col_' + str(amount_features + 1)  # Target: the first pixel in the board
target = reverse_df_sort[name_col]

# Step 2: Split Data
X_train_val, X_test, y_train_val, y_test = train_test_split(features, target, test_size=0.1, random_state=365)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1, random_state=365)

print("len x train: ", len(X_train))
print("len x test: ",len(X_test))
print("len y train: ",len(y_train))
print("len y test: ",len(y_test))

len x train:  33506
len x test:  4137
len y train:  33506
len y test:  4137


In [7]:
X_train.shape

(33506, 100)

In [8]:
X_train_array = X_train.to_numpy()
y_train_array = y_train.to_numpy()

In [9]:
print(X_train_array.shape)
print(y_train_array.shape)

(33506, 100)
(33506,)


In [10]:
X_train_array = X_train_array.reshape((X_train.shape[0],SIZE,SIZE,1))
y_train_array = y_train_array.reshape((y_train.shape[0],1))

In [11]:
print(X_train_array.shape)
print(y_train_array.shape)

(33506, 10, 10, 1)
(33506, 1)


In [12]:
X_val_array = X_val.to_numpy()
X_val_array = X_val_array.reshape((X_val.shape[0],SIZE,SIZE,1))
y_val_array = y_val.to_numpy()
y_val_array = y_val_array.reshape((y_val.shape[0],1))

X_test_array = X_test.to_numpy()
X_test_array = X_test_array.reshape((X_test.shape[0],SIZE,SIZE,1))
y_test_array = y_test.to_numpy()
y_test_array = y_test_array.reshape((y_test.shape[0],1))

In [13]:
import tensorflow as tf
import numpy as np

# --- פרמטרים ---
SIZE = 10             # גודל הלוח
gen_data = gen - 1    # מספר הלוחות הרציפים בקלט
BATCH_SIZE = 32
EPOCHS = 3

# --- PREPROCESSING ---
# X_train_array.shape = (num_samples + gen_data, SIZE, SIZE, 1)
# y_train_array.shape = (num_samples, 1)  ← תא אחד בלבד (0 או 1)

num_samples = X_train_array.shape[0] - gen_data

X_train = np.zeros((num_samples, gen_data, SIZE, SIZE, 1), dtype='float32')
y_train = np.zeros((num_samples, 1), dtype='float32')  # רק תא אחד

for i in range(num_samples):
    X_train[i] = X_train_array[i:i+gen_data]   # רצף של gen_data לוחות
    y_train[i] = y_train_array[i]              # הפלט: תא אחד (0/1)

print("X_train shape:", X_train.shape)  # (num_samples, gen_data, SIZE, SIZE, 1)
print("y_train shape:", y_train.shape)  # (num_samples, 1)

# --- MODEL ---
model = tf.keras.Sequential([
    tf.keras.layers.ConvLSTM2D(
        filters=32,
        kernel_size=(3,3),
        activation='relu',
        padding='same',
        return_sequences=True,
        input_shape=(gen_data, SIZE, SIZE, 1)
    ),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.ConvLSTM2D(
        filters=64,
        kernel_size=(3,3),
        activation='relu',
        padding='same',
        return_sequences=False
    ),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # ← פלט יחיד בינארי
])

X_train shape: (33505, 1, 10, 10, 1)
y_train shape: (33505, 1)


c:\Users\דרור\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [14]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv_lstm2d (ConvLSTM2D)        │ (None, 1, 10, 10, 32)  │        38,144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 1, 10, 10, 32)  │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_lstm2d_1 (ConvLSTM2D)      │ (None, 10, 10, 64)     │       221,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 10, 10, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 6400)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │       409,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 669,697 (2.55 MB)

 Trainable params: 669,505 (2.55 MB)

 Non-trainable params: 192 (768.00 B)

In [15]:
# אימון
history = model.fit(
    X_train, y_train,
    epochs=3,
    batch_size=32,
    validation_split=0.2,
    shuffle=True
)

Epoch 1/3
838/838 ━━━━━━━━━━━━━━━━━━━━ 121s 104ms/step - accuracy: 0.8042 - loss: 0.4103 - val_accuracy: 0.8320 - val_loss: 0.3569
Epoch 2/3
838/838 ━━━━━━━━━━━━━━━━━━━━ 123s 82ms/step - accuracy: 0.8535 - loss: 0.3261 - val_accuracy: 0.8388 - val_loss: 0.3611
Epoch 3/3
838/838 ━━━━━━━━━━━━━━━━━━━━ 82s 81ms/step - accuracy: 0.8826 - loss: 0.2733 - val_accuracy: 0.8339 - val_loss: 0.3628


In [16]:
X_test = X_test_array.reshape((-1, gen_data, SIZE, SIZE, 1)).astype('float32')
y_test = y_test_array.astype('float32')

test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_acc:.3f}")

130/130 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - accuracy: 0.8191 - loss: 0.3911
Test accuracy: 0.831


In [17]:
import numpy as np
from sklearn.metrics import confusion_matrix

def evaluate_model(model, X_test_array, y_test_array, gen_data, SIZE):
    """
    מעריך את ביצועי המודל ומציג טבלת Confusion Matrix ומדדים מסוכמים.
    """
    # עיבוד הנתונים
    X_test = X_test_array.reshape((-1, gen_data, SIZE, SIZE, 1)).astype('float32')
    y_test = y_test_array.reshape((-1, 1)).astype('float32')

    # חיזוי
    y_pred = model.predict(X_test)
    y_pred_binary = (y_pred > 0.5).astype(int)

    # Confusion matrix
    cm = confusion_matrix(y_test, y_pred_binary)
    tn, fp, fn, tp = cm.ravel()

    # חישוב מדדים
    precision = tp / (tp + fp + 1e-8)
    recall = tp / (tp + fn + 1e-8)
    f1 = 2 * (precision * recall) / (precision + recall + 1e-8)
    acc = (tp + tn) / (tp + tn + fp + fn)

    # --- הדפסה בטבלה ---
    print("\n===== Evaluation Results =====")
    print("┌──────────────┬────────────┬────────────┐")
    print(f"│              │ Pred=Alive │ Pred=Dead  │")
    print("├──────────────┼────────────┼────────────┤")
    print(f"│ True=Alive   │ {tp:10d} │ {fn:10d} │")
    print(f"│ True=Dead    │ {fp:10d} │ {tn:10d} │")
    print("└──────────────┴────────────┴────────────┘")

    print("\n--- Performance Metrics ---")
    print(f"{'Accuracy':<12}: {acc:.3f}")
    print(f"{'Precision':<12}: {precision:.3f}")
    print(f"{'Recall':<12}: {recall:.3f}")
    print(f"{'F1-score':<12}: {f1:.3f}")


In [18]:
# יצירת סט בדיקה
num_samples_test = X_test_array.shape[0] - gen_data
X_test = np.zeros((num_samples_test, gen_data, SIZE, SIZE, 1), dtype='float32')
y_test = np.zeros((num_samples_test, 1), dtype='float32')

for i in range(num_samples_test):
    X_test[i] = X_test_array[i:i+gen_data]
    y_test[i] = y_test_array[i]

# הפעלת פונקציית ההערכה
results = evaluate_model(model, X_test, y_test, gen_data, SIZE)


130/130 ━━━━━━━━━━━━━━━━━━━━ 5s 29ms/step

===== Evaluation Results =====
┌──────────────┬────────────┬────────────┐
│              │ Pred=Alive │ Pred=Dead  │
├──────────────┼────────────┼────────────┤
│ True=Alive   │        532 │        410 │
│ True=Dead    │        291 │       2903 │
└──────────────┴────────────┴────────────┘

--- Performance Metrics ---
Accuracy    : 0.831
Precision   : 0.646
Recall      : 0.565
F1-score    : 0.603
